In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob
from subprocess import check_output

import yaml
import dask

import util

USER = os.environ['USER']

assert os.path.exists('/glade/campaign'), (
    'campaign is not accessible; run on Casper'
)

In [3]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore") # Silence warnings
#import hvplot.xarray
import xarray as xr
import cartopy.crs as ccrs
import numpy as np
import math
from matplotlib.pyplot import figure

# MatPlotlib
import matplotlib.pyplot as plt
from matplotlib import pylab

# Scientific libraries
from numpy import arange,array,ones
from scipy import stats

import seaborn as sns
import scipy.io as sio

import numpy.ma as ma
from netCDF4 import Dataset as NetCDFFile
import glob
import pylab 
from copy import deepcopy
import pandas as pd
import os
import matplotlib.gridspec as gridspec
import matplotlib
import cartopy
import matplotlib.colors as colors
import cmocean
from cartopy.util import add_cyclic_point

In [4]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [5]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        if clip is None:
            clip = self.clip

        result, is_scalar = self.process_value(value)

        self.autoscale_None(result)
        vmin, vmax, midpoint = self.vmin, self.vmax, self.midpoint

        if not (vmin < midpoint < vmax):
            raise ValueError("midpoint must be between maxvalue and minvalue.")
        elif vmin == vmax:
            result.fill(0) # Or should it be all masked? Or 0.5?
        elif vmin > vmax:
            raise ValueError("maxvalue must be bigger than minvalue")
        else:
            vmin = float(vmin)
            vmax = float(vmax)
            if clip:
                mask = np.ma.getmask(result)
                result = np.ma.array(np.clip(result.filled(vmax), vmin, vmax),
                                  mask=mask)

            # ma division is very slow; we can take a shortcut
            resdat = result.data

            #First scale to -1 to 1 range, than to from 0 to 1.
            resdat -= midpoint
            resdat[resdat>0] /= abs(vmax - midpoint)
            resdat[resdat<0] /= abs(vmin - midpoint)

            resdat /= 2.
            resdat += 0.5
            result = np.ma.array(resdat, mask=result.mask, copy=False)

        if is_scalar:
            result = result[0]
        return result


In [6]:
def adjust_pop_grid(tlon,tlat,field):
    nj = tlon.shape[0]
    ni = tlon.shape[1]
    xL = int(ni/2 - 1)
    xR = int(xL + ni)

    tlon = np.where(np.greater_equal(tlon,np.min(tlon[:,0])),tlon-360.,tlon)
    lon  = np.concatenate((tlon,tlon+360.),1)
    lon = lon[:,xL:xR]

    if ni == 320:
        lon[367:-3,0] = lon[367:-3,0]+360.
    lon = lon - 360.
    lon = np.hstack((lon,lon[:,0:1]+360.))
    if ni == 320:
        lon[367:,-1] = lon[367:,-1] - 360.

    #-- trick cartopy into doing the right thing:
    #   it gets confused when the cyclic coords are identical
    lon[:,0] = lon[:,0]-1e-8

    #-- periodicity
    lat  = np.concatenate((tlat,tlat),1)
    lat = lat[:,xL:xR]
    lat = np.hstack((lat,lat[:,0:1]))

    field = np.ma.concatenate((field,field),1)
    field = field[:,xL:xR]
    field = np.ma.hstack((field,field[:,0:1]))
    return lon,lat,field

In [7]:
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [8]:
path = '/glade/campaign/cesm/development/bgcwg/projects/marbl-spectra/GNG595_2ndcycle_1990-2009_clim'
files = sorted(glob.glob(f'{path}/*.nc', recursive=True),key=numericalSort)
ds= xr.open_mfdataset(files,combine='by_coords')

In [9]:
cluster, client = util.get_ClusterClient(walltime='24:00:00')
cluster.scale(32)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gabyn/proxy/42600/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gabyn/proxy/42600/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.46:46104,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gabyn/proxy/42600/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [9]:
case = 'g.e21.G1850ECOIAF.t62_g17.marbl0_33.GNG595'
path = f'/glade/campaign/cesm/development/bgcwg/projects/marbl-spectra/{case}/ocn/hist'

variables = [f'{var}' for var in ['graze_diaz_zoo1','graze_diaz_zoo2','graze_diaz_zoo3',
                                  'graze_pp_zoo1',
                                  'graze_mp1_zoo1','graze_mp1_zoo2',
                                  'graze_mp2_zoo2','graze_mp2_zoo3','graze_mp2_zoo4',
                                  'graze_mp3_zoo3','graze_mp3_zoo4','graze_mp3_zoo5',
                                  'graze_mp4_zoo4','graze_mp4_zoo5','graze_mp4_zoo6',
                                  'graze_diat1_zoo1','graze_diat1_zoo2','graze_diat1_zoo3','graze_diat1_zoo4',
                                  'graze_diat2_zoo2','graze_diat2_zoo3','graze_diat2_zoo4','graze_diat2_zoo5',
                                  'graze_diat3_zoo3','graze_diat3_zoo4','graze_diat3_zoo5','graze_diat3_zoo6',
                                  'graze_zoo1_zoo2',
                                  'graze_zoo1_zoo3',
                                  'graze_zoo2_zoo3',
                                  'graze_zoo2_zoo4',
                                  'graze_zoo3_zoo4',
                                  'graze_zoo3_zoo5',
                                  'graze_zoo4_zoo5',
                                  'graze_zoo4_zoo6',
                                  'graze_zoo5_zoo6',
                                 ]]
coords = {'x':'TLONG','y':'TLAT'}
keep_vars = variables + list(coords.values())+['dz','KMT','time']

In [10]:
%%time
ds_fall_avg = xr.Dataset()

for year in np.arange(63,125,1):
    yr4="0{:02d}".format(year).zfill(4)
    print(year)
    
    ds_fall = xr.Dataset()

    file = sorted(glob.glob(f'{path}/{case}.pop.h.{yr4}-*.nc'))
    dsv_fall=xr.open_mfdataset(file[8:11], decode_times=True,drop_variables=["transport_components", "transport_regions"], 
                            parallel=True, compat="override", combine='nested', concat_dim="time",data_vars="minimal",coords='minimal' )

    for vv in variables: 
        ds_fall = xr.merge((ds_fall, dsv_fall[vv]))
    
    ds_fall = ds_fall.drop([v for v in ds_fall.variables if v not in keep_vars]).squeeze()
    ds_fall = ds_fall.mean(dim='time')
    ds_fall_avg = xr.concat([ds_fall_avg, ds_fall],dim='year')

63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
CPU times: user 3min 57s, sys: 4min 23s, total: 8min 20s
Wall time: 11min 59s


In [11]:
## Gaby's Path 
jluo_path = '/glade/u/home/jluo/cesm_scripts/size_structured/nl_input/cases/g.e21.G1850ECOIAF.t62_g17.marbl0_33.'
gabyn_path = '/glade/work/gabyn/case_notes/spectral_cases/g.e21.G1850ECOIAF.t62_g17.marbl0_33.'
nl_config = os.path.join(''+gabyn_path+'GNG595/data/')
sizes = pd.read_csv(nl_config+'plankton_sizes.csv')
sizes = sizes.sort_values('mmolC')
sizes

,type,sname,mass_ugC,vol_um3,ESD_mm,Qp_fixed,mmolC
1,phyto,pp,9.266989e-08,3.706795e-01,0.000891,0.004651,7.715418e-12
2,phyto,mp1,8.241979e-06,1.805089e+01,0.003255,0.006824,6.862025e-10
0,phyto,diaz,5.499929e-05,1.227865e+02,0.006167,0.003333,4.579077e-09
3,phyto,mp2,8.888712e-05,8.790197e+02,0.011885,0.008656,7.400476e-09
6,phyto,diat1,5.004513e-04,1.327390e+04,0.029376,0.010289,4.166608e-08
9,zoo,zoo1,2.803686e-03,2.355042e+04,0.035563,0.008547,2.334265e-07
7,phyto,diat2,3.803133e-03,1.326626e+05,0.063278,0.012602,3.166375e-07
4,phyto,mp3,4.830334e-03,4.280541e+04,0.043401,0.012907,4.021592e-07
8,phyto,diat3,2.890156e-02,1.325863e+06,0.136301,0.015435,2.406258e-06
10,zoo,zoo2,8.933984e-02,7.447577e+05,0.112462,0.008547,7.438168e-06


In [12]:
prey_varnames = {}
for i,v in enumerate(sizes.sname):
    prey_varnames[v]=i

In [13]:
pred_varnames = {'zoo1':0,'zoo2':1,'zoo3':2,'zoo4':3,'zoo5':4,'zoo6':5}
grazing_vars = [v for v in ds if 'graze' in v and 'zint' not in v]

In [14]:
graze_split = [v.split('_',3) for v in grazing_vars]
graze_split = pd.DataFrame(graze_split)
graze_split = graze_split.rename({0:'var',1:'prey',2:'pred'}, axis=1)
graze_split['var'] = grazing_vars

In [15]:
zoo_names = { 'zoo_names': ['zoo1','zoo2','zoo3','zoo4','zoo5','zoo6']}
zn = pd.DataFrame(data=zoo_names)

phyto_names = { 'phyto_names': ['pp','mp1','diaz','mp2','diat1','diat2','mp3','diat3','mp4']}
pn = pd.DataFrame(data=phyto_names)

In [16]:
# set trophic level for phyto = 1
tmpOnes = np.ones(shape=ds_fall_avg.graze_diat3_zoo5.shape)
tlvals = {}
for v in pn.phyto_names:
    tlvals[v]=tmpOnes

# set trophic level for zoo = NaN initially
tmp = tmpOnes.copy()
tmp[:] = np.nan
for v in zn.zoo_names:
    tlvals[v]=tmp
    
zoo = ['zoo1','zoo2','zoo3','zoo4','zoo5','zoo6']

In [17]:
for i,vs in enumerate(zn.zoo_names):
    
    # pull out the appropriate grazing variables in the history file
    graze_vars = [v for v in ds_fall_avg if re.match('graze_(.*)_'+vs,v)]
    
    # holder for the grazing values
    shape = ds_fall_avg[graze_vars[0]].shape + (len(graze_vars),)

    graze_vals = np.empty(shape=shape)
    graze_tl = np.ones(shape=shape)
    
    for gi,gv in enumerate(graze_vars):
        
        # pull out identity of prey
        g,prey,pred = gv.split('_',3)
        print(prey,pred)
        
        # if prey is zooplankton, then apply alternate trophic level
        if prey in zoo:
            graze_tl[...,gi]=tlvals[prey]
        
        graze_vals[...,gi] = ds_fall_avg[gv].values # Ellipses select the last slice
    # graze_vals*grazetl/graze_vals    
    tlvals[vs] = 1 + (np.nansum(graze_vals * graze_tl, axis=-1) / np.nansum(graze_vals, axis=-1)) # axis=-1 means sum over the last axis

diaz zoo1
pp zoo1
mp1 zoo1
diat1 zoo1
diaz zoo2
mp1 zoo2
mp2 zoo2
diat1 zoo2
diat2 zoo2
zoo1 zoo2
diaz zoo3
mp2 zoo3
mp3 zoo3
diat1 zoo3
diat2 zoo3
diat3 zoo3
zoo1 zoo3
zoo2 zoo3
mp2 zoo4
mp3 zoo4
mp4 zoo4
diat1 zoo4
diat2 zoo4
diat3 zoo4
zoo2 zoo4
zoo3 zoo4
mp3 zoo5
mp4 zoo5
diat2 zoo5
diat3 zoo5
zoo3 zoo5
zoo4 zoo5
mp4 zoo6
diat3 zoo6
zoo4 zoo6
zoo5 zoo6


In [18]:
%%time
zoods = xr.DataArray(tlvals['zoo1'], coords={'year':ds_fall_avg.year, 'z_t_150m':ds.z_t_150m, 'nlat':ds.nlat, 'nlon':ds.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])
zoods = zoods.to_dataset(name='zoo1TL_fall')
zoods['zoo2TL_fall'] = xr.DataArray(tlvals['zoo2'], coords={'year':ds_fall_avg.year, 'z_t_150m':ds.z_t_150m, 'nlat':ds.nlat, 'nlon':ds.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])
zoods['zoo3TL_fall'] = xr.DataArray(tlvals['zoo3'], coords={'year':ds_fall_avg.year, 'z_t_150m':ds.z_t_150m, 'nlat':ds.nlat, 'nlon':ds.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])
zoods['zoo4TL_fall'] = xr.DataArray(tlvals['zoo4'], coords={'year':ds_fall_avg.year, 'z_t_150m':ds.z_t_150m, 'nlat':ds.nlat, 'nlon':ds.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])
zoods['zoo5TL_fall'] = xr.DataArray(tlvals['zoo5'], coords={'year':ds_fall_avg.year, 'z_t_150m':ds.z_t_150m, 'nlat':ds.nlat, 'nlon':ds.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])
zoods['zoo6TL_fall'] = xr.DataArray(tlvals['zoo6'], coords={'year':ds_fall_avg.year, 'z_t_150m':ds.z_t_150m, 'nlat':ds.nlat, 'nlon':ds.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])

CPU times: user 9.35 ms, sys: 0 ns, total: 9.35 ms
Wall time: 9.36 ms


In [19]:
np.nanmax(tlvals['zoo6'])

5.99589356034772

In [20]:
%%time
zoods['zooTL_fall'] = xr.concat([zoods[v+'TL_fall'] for v in ['zoo1','zoo2','zoo3','zoo4','zoo5','zoo6']],dim='var').mean(dim='var')

CPU times: user 5.15 s, sys: 5.02 s, total: 10.2 s
Wall time: 10.5 s


In [21]:
zoods.to_netcdf('/glade/derecho/scratch/gabyn/SPECTRA_hindcast/SPECTRA_1990_2009/GNG595_monthly_yearly_1948_2009/zooTL_62yr_fall.nc');

In [23]:
client.close()
cluster.close()